# TextMining Assignment 2
Lucas de Wolff (s3672980) and Ruben Ahrens (s3677532)

November 2023


In [1]:
import datasets
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from huggingface_hub import interpreter_login
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [2]:
names = ['O','B-adr', 'B-disease', 'B-drug', 'B-symptom', 'B-finding','I-adr', 'I-disease', 'I-drug', 'I-symptom', 'I-finding']

In [3]:
train, valid, test = 'train.ann', 'valid.ann', 'test.ann'
path = '../../cadec/processed/'

raw_datasets = {}
for file, name in zip([train, valid, test], ['train', 'validation', 'test']):
    id = 0
    raw_datasets[name] = {'id': [], 'tokens': [], 'ner_tags': []}
    with open(f'{path}/{file}', 'r') as f:
        tokens, ner_tags = [], []
        for line in f:
            try:
                token, ner_tag = line.split()
                tokens.append(token)
                ner_tags.append(names.index(ner_tag))
            except:
                raw_datasets[name]['id'].append([id for _ in range(len(tokens))])
                raw_datasets[name]['tokens'].append(tokens)
                raw_datasets[name]['ner_tags'].append(ner_tags)
                id += 1
                tokens, ner_tags = [], []
    raw_datasets[name] = datasets.Dataset.from_dict(raw_datasets[name])

raw_datasets = datasets.DatasetDict(raw_datasets)
display(raw_datasets)
# # load datasetdict from disk
# raw_datasets = datasets.load_from_disk('../../cadec/processed')
# display(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 988
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 185
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 132
    })
})

# Processing the data

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
tokenizer.is_fast

True

In [6]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(raw_datasets["train"]["tokens"])
print(inputs)

[['no', 'side', 'effects', 'for', 'the', 'first', 'two', 'months', '.', 'then', 'va', '##ginal', 'bleeding', '2', 'w', '##ks', 'after', 'men', '##stra', '##l', 'cycle', '.', 'stomach', 'pain', '.', 'can', '##ker', 'sore', '##s', 'in', 'my', 'mouth', '.', 'headache', '.', 'been', 'off', 'for', '1', 'week', 'still', 'have', 'bleeding', '.', 'helped', 'my', 'pain', 'al', '##ot', '.', 'too', 'scared', 'to', 'take', 'this', 'drug', 'again', '.'], ['1st', 'pill', 'taken', 'with', 'food', ',', 'a', 'few', 'hours', 'after', 'i', 'experienced', 'short', '##ness', 'of', 'breath', ',', 'a', 'sense', 'of', 'depression', ',', 'cr', '##amp', '##ing', ',', 'upset', 'stomach', '##e', 'will', 'stop', 'taking', 'immediately', '.', 'honestly', 'can', 'not', 'recommend', 'this', 'drug', ',', 'i', 'am', 'still', 'experiencing', 'side', 'effects', 'just', 'after', '1', 'pill', 'and', 'im', 'supposed', 'to', 'take', 'twice', '/', 'day', '75', '##mg', '.', 'i', 'just', 'can', '##t', 'keep', 'going', 'have', '

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [8]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(len(labels), len(word_ids))
print(align_labels_with_tokens(labels, word_ids))

58 80
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 6, 0, 1, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [10]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

# compare total number of tokens before and after tokenization. which is not the number of rows, but the number of tokens
train_set = set([word  for sentence  in  raw_datasets['train']['tokens'] for word in sentence ])
eval_set = set([word  for  sentence  in raw_datasets['validation']['tokens'] for word in sentence ])
test_set = set([word  for  sentence  in raw_datasets['test']['tokens'] for word in sentence ])

print(len(train_set), len(eval_set), len(test_set))

# Print unique tokens of each dataset
train_set_unique = train_set - eval_set - test_set
eval_set_unique = eval_set - train_set - test_set
test_set_unique = test_set - train_set - eval_set

print(len(train_set_unique), len(eval_set_unique), len(test_set_unique))

Map:   0%|          | 0/988 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

5629 2247 2240
2983 271 270


# Fine-tuning the model with keras

## Data collation

In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Metrics

In [12]:
metric = evaluate.load("seqeval")

In [13]:

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [14]:
def compute_all_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return all_metrics

In [15]:
def write_tex_table(all_metrics, filename):
    with open(filename, 'w') as f:
        f.write("\\begin{table}[h]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{lrrrr}\n")
        f.write("\\toprule\n")
        f.write(" & \\textbf{Precision} & \\textbf{Recall} & \\textbf{F1} & \\textbf{Number} \\\\\n")
        f.write("\\midrule\n")
        for key, value in all_metrics.items():
            if "overall" not in key:
                f.write(f"{key.title()} & {value['precision']:.2f} & {value['recall']:.2f} & {value['f1']:.2f} & {value['number']} \\\\\n")
        try:
            f.write("\\midrule\n")
            f.write(" & \\textbf{Overall precision} & \\textbf{Overall recall} & \\textbf{Overall F1} & \\textbf{Overall accuracy} \\\\\n")
            f.write("\\midrule\n")
            f.write(f" & {all_metrics['overall_precision']:.2f} & {all_metrics['overall_recall']:.2f} & {all_metrics['overall_f1']:.2f} & {all_metrics['overall_accuracy']:.2f} \\\\\n")
        except:
            pass
        f.write("\\bottomrule\n")
        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")
    
def print_fancy_table(all_metrics):
    # display fancy table in ipython widget
    from IPython.display import display, HTML
    html = "<table>"
    html += "<tr><th></th><th>Precision</th><th>Recall</th><th>F1</th><th>Number</th></tr>"
    for key, value in all_metrics.items():
        if "overall" not in key:
            html += f"<tr><td>{key.title()}</td><td>{value['precision']:.2f}</td><td>{value['recall']:.2f}</td><td>{value['f1']:.2f}</td><td>{value['number']}</td></tr>"
    html += "<tr><th></th><th>Overall precision</th><th>Overall recall</th><th>Overall F1</th><th>Overall accuracy</th></tr>"
    html += f"<tr><td></td><td>{all_metrics['overall_precision']:.2f}</td><td>{all_metrics['overall_recall']:.2f}</td><td>{all_metrics['overall_f1']:.2f}</td><td>{all_metrics['overall_accuracy']:.2f}</td></tr>"
    html += "</table>"
    display(HTML(html))
    

## Defining the model

In [16]:
id2label = {i: label for i, label in enumerate(names)}
label2id = {v: k for k, v in id2label.items()}

In [17]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##  Fine-tuning the model

In [18]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.
Token is valid (permission: write).
Your token has been saved in your con

## Hyperparameter optimization

In [95]:
training_args = TrainingArguments(
    "bert-finetuned-ner-optuna",
    evaluation_strategy="epoch",
    logging_strategy="no",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

def model_init(): 
    return AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)    

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "adam_beta1": trial.suggest_float("beta_1", 0.9, 0.999),
        "adam_beta2": trial.suggest_float("beta_2", 0.9, 0.999),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True),
    }

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    model_init=model_init,
)

best_run = trainer.hyperparameter_search(
    direction="minimize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=2,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2023-12-19 15:26:25,698] A new study created in memory with name: no-name-001bdacc-bdbc-44d7-b758-fac90408dc11
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.7918012738227844, 'eval_precision': 0.0012903225806451613, 'eval_recall': 0.00089126559714795, 'eval_f1': 0.0010542962572482868, 'eval_accuracy': 0.7313527582047864, 'eval_runtime': 14.3197, 'eval_samples_per_second': 8.24, 'eval_steps_per_second': 1.048, 'epoch': 1.0}


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.5295488238334656, 'eval_precision': 0.3015753938484621, 'eval_recall': 0.3582887700534759, 'eval_f1': 0.3274949083503055, 'eval_accuracy': 0.829556274995239, 'eval_runtime': 12.9165, 'eval_samples_per_second': 9.136, 'eval_steps_per_second': 1.161, 'epoch': 2.0}


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-12-19 17:11:47,366] Trial 1 finished with value: 1.8773719613667539 and parameters: {'learning_rate': 1.614305313593754e-05, 'per_device_train_batch_size': 32, 'beta_1': 0.9191188381394011, 'beta_2': 0.9641724268127546, 'weight_decay': 0.09440442683896338, 'adam_epsilon': 7.062980816400544e-08}. Best is trial 0 with value: 1.7387980701188146.


{'eval_loss': 0.48549458384513855, 'eval_precision': 0.3199691595990748, 'eval_recall': 0.3698752228163993, 'eval_f1': 0.34311699049193883, 'eval_accuracy': 0.844410588459341, 'eval_runtime': 16.8766, 'eval_samples_per_second': 6.992, 'eval_steps_per_second': 0.889, 'epoch': 3.0}
{'train_runtime': 3896.6356, 'train_samples_per_second': 0.723, 'train_steps_per_second': 0.023, 'train_loss': 0.7529549492730034, 'epoch': 3.0}


## Retrain model with optimized hyperparameters

In [23]:
training_args = TrainingArguments(
    "bert-finetuned-ner-optuna",
    evaluation_strategy="epoch",
    logging_strategy="no",
    save_strategy="no",
    learning_rate = 1.2753679840290149e-05,
    per_device_train_batch_size = 32,
    adam_beta1 = 0.9288343770865429,
    adam_beta2 = 0.9128142694246015,
    weight_decay = 0.05090189622476768,
    adam_epsilon = 2.3198092141075746e-08,
    num_train_epochs=10,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# for n, v in best_run.hyperparameters.items():
#     print(f"{n}: {v}")
#     setattr(trainer.args, n, v)
    
trainer.train()
trainer.push_to_hub(commit_message="Training complete")

  0%|          | 0/310 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/24 [00:00<?, ?it/s]

c:\Users\Ruben\miniconda3\envs\torch\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.7362387776374817, 'eval_precision': 0.04236200256739409, 'eval_recall': 0.023305084745762712, 'eval_f1': 0.030068337129840545, 'eval_accuracy': 0.7665100707381608, 'eval_runtime': 3.2781, 'eval_samples_per_second': 56.436, 'eval_steps_per_second': 7.321, 'epoch': 1.0}


...

{'eval_loss': 0.32383793592453003, 'eval_precision': 0.5163660654642619, 'eval_recall': 0.5459039548022598, 'eval_f1': 0.5307243391692413, 'eval_accuracy': 0.8995626113721044, 'eval_runtime': 2.967, 'eval_samples_per_second': 62.353, 'eval_steps_per_second': 8.089, 'epoch': 10.0}
{'train_runtime': 5160.9242, 'train_samples_per_second': 1.914, 'train_steps_per_second': 0.06, 'train_loss': 0.41906039330267136, 'epoch': 10.0}


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

'https://huggingface.co/rubinho/bert-finetuned-ner-optuna/tree/main/'

In [24]:
predictions = trainer.predict(tokenized_datasets["test"])
eval_preds = predictions.predictions, predictions.label_ids

  0%|          | 0/17 [00:00<?, ?it/s]

In [25]:
all_metrics = compute_all_metrics(eval_preds)
logits, labels = eval_preds
predictions = np.argmax(logits, axis=-1)

# Remove ignored index (special tokens) and convert to labels
true_labels = [[names[l] for l in label if l != -100] for label in labels]
true_predictions = [
    [names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

from seqeval.metrics import classification_report
print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

         adr       0.43      0.56      0.49       800
     disease       0.59      0.31      0.41       171
        drug       0.62      0.58      0.60       217
     finding       0.05      0.02      0.02        60
     symptom       0.67      0.03      0.05        80

   micro avg       0.46      0.47      0.47      1328
   macro avg       0.47      0.30      0.31      1328
weighted avg       0.48      0.47      0.45      1328



In [26]:
# flatten the labels and predictions
true_labels_flat = [item for sublist in true_labels for item in sublist]
true_predictions_flat = [item for sublist in true_predictions for item in sublist]

# compute precision, recall, and f1 per entity type
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, true_predictions_flat, average=None, labels=names[1:])

# count number of entities per type
from collections import Counter
count = Counter(true_labels_flat)

# format into pandas table and round to 2 decimals
import pandas as pd
df = pd.DataFrame({'precision': precision, 'recall': recall, 'f1': f1, 'number': count}, index=names[1:])
df = df.round(2)
display(df)
write_tex_table(df.to_dict('index'), 'bert-finetuned-ner_optuna.txt')

c:\Users\Ruben\miniconda3\envs\torch\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1,number
B-adr,0.68,0.74,0.71,733
B-disease,0.69,0.47,0.56,132
B-drug,0.79,0.92,0.85,143
B-symptom,0.67,0.03,0.06,67
B-finding,0.00,0.00,0.00,37
I-adr,0.73,0.79,0.76,2378
I-disease,0.00,0.00,0.00,70
I-drug,0.82,0.79,0.81,884
I-symptom,0.00,0.00,0.00,60
I-finding,0.27,0.09,0.13,196
